In [9]:
#source:https://github.com/dwavesystems/dwave-hybrid which itself is licensed under Apache License 2.0
import dimod
import hybrid
import networkx as nx
import scipy
import dwave_networkx as dnx
import numpy as np

# This function performs the max clique operation but uses the hybrid instead of the max_clique function in ocean SDK
def hybrid_max_clique(input_qubo, probsize):
    # Construct a problem
    bqm = dimod.BinaryQuadraticModel.from_qubo(input_qubo)
    #bqm = dimod.BinaryQuadraticModel({}, input_qubo, 0, dimod.SPIN)
    # definition of workflow - choosing to use hybrid workflows instead of the max_clique solver 
    #5 racing branches are defines, the interruptible tabu, and combination of 2 samplers(auto, simulator) and 2 decomposers (energy impact, random subproblem) )
    iteration = hybrid.RacingBranches(
        hybrid.InterruptableTabuSampler(),
        hybrid.EnergyImpactDecomposer(size=probsize)
        | hybrid.QPUSubproblemAutoEmbeddingSampler()
        | hybrid.SplatComposer(),
        hybrid.EnergyImpactDecomposer(size=probsize)
        | hybrid.SimulatedAnnealingSubproblemSampler()
        | hybrid.SplatComposer(),
        hybrid.RandomSubproblemDecomposer(size=probsize)
        | hybrid.QPUSubproblemAutoEmbeddingSampler()
        | hybrid.SplatComposer(),
        hybrid.RandomSubproblemDecomposer(size=probsize)
        | hybrid.SimulatedAnnealingSubproblemSampler()
        | hybrid.SplatComposer(),
    ) | hybrid.ArgMin()
    workflow = hybrid.LoopUntilNoImprovement(iteration, convergence=3)
    # Solve the problem
    init_state = hybrid.State.from_problem(bqm)
    final_state = workflow.run(init_state).result()
    # Print results
    print("Solution: sample={.samples.first}".format(final_state))
    
    return final_state


##Sample array - hydrogen atom - source:https://github.com/Qiskit/qiskit-aqua
G=np.array([
    [0, 1, 1, 1, 1],
    [1, 0, 1, 1, 0],
    [1, 1, 0, 1, 0],
    [1, 1, 1, 0, 0],
    [1, 0, 0, 0, 0]
])

## NOTE - problem_size must be size of the input matrix
problem_size=len(G)

##runs the code
graph = nx.Graph(G)
graph_comp = nx.complement(graph)
input_hammer=dnx.algorithms.independent_set.maximum_weighted_independent_set_qubo(graph_comp)
hybrid_max_clique(input_hammer,problem_size)

Solution: sample=Sample(sample={0: 1, 1: 1, 2: 1, 3: 1, 4: 0}, energy=-4.0, num_occurrences=1)


{'problem': BinaryQuadraticModel({0: -1.0, 1: -1.0, 2: -1.0, 3: -1.0, 4: -1.0}, {(1, 4): 2.0, (2, 4): 2.0, (3, 4): 2.0}, 0.0, Vartype.BINARY),
 'samples': SampleSet(rec.array([([1, 1, 1, 1, 0], -4., 1)],
           dtype=[('sample', 'i1', (5,)), ('energy', '<f8'), ('num_occurrences', '<i4')]), [0, 1, 2, 3, 4], {}, 'BINARY')}